In [12]:
import pandas as pd
import re

In [13]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

In [14]:
df = pd.read_excel('news_label.xlsx', engine='openpyxl')

In [15]:
df = df.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
df['content'] = df['title'] + df['content']

In [16]:
df = df.drop(columns=['근거', 'title'])

In [17]:
df['label'] = df['label'].astype(int)
df

,date,content,label
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...,1
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",1
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...,1
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\...",1
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",1
...,...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는...",0
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",1
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르...",0
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...,0


### 데이터 정제하기

In [18]:
# 문자 외의 것들 제거
df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
# 혹시 모를 다중 공백 제거
df['content'] = df['content'].str.replace(' +',' ') 
df

/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_2109/2956849729.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_2109/2956849729.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(' +',' ')


,date,content,label
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...,1
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...,1
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...,1
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...,1
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...,1
...,...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...,0
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...,1
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...,0
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...,0


from sklearn.model_selection import train_test_split
X = df['content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

X_train

X_test

### 토큰화 및 TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화

### 감성 분석 모델 구축

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


def get_pipe(model, model_name: str) -> Pipeline:
    "TfidfVectorizer와 모델을 연결한 파이프라인을 반환하는 함수"
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 3))
    pipe = Pipeline([
        ("tfidf", tfidf),
        (model_name, model)
    ])
    return pipe

# 그냥 train 데이터를 tfidf로 fit_transform한 데이터를 만듭시다...

In [20]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def return_kfold_accuarcy(model, k: int = 5) -> float:
    global train
    "모델을 입력받아 KFold 예측 후 accuracy score를 반환하는 함수"
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    for train_idx, test_idx in kfold.split(df["content"], df["label"]):
        train, val = df.iloc[train_idx], df.iloc[test_idx]
        model.fit(train["content"], train["label"])
        pred = model.predict(val["content"])
        acc = accuracy_score(val["label"], pred)
        result.append(acc)

    return np.mean(result)

In [27]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC


models = [
    ("naive_bayes", BernoulliNB()),
    ("naive_bayes", BernoulliNB(alpha=1, fit_prior=True)),
#     ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
#     ("rfc", RandomForestClassifier(random_state=42, n_jobs=-1)),
#     ("SVC", SVC(random_state=42)),
#     ("ada", AdaBoostClassifier(random_state=42))
]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

# hyper parameter 더 수정해보기 (어떤 의미인지?)

# cross validation - best parameter를 찾아주는 것 (함수)
# optuna, AutoML
# 파라미터 튜닝해서 해보기
# 전이학습(kcbert) / 우리의 test data에 알맞게 데이터를 바꾸는 것(fine tuning)
# fine tuning을 하기 위해 monitoring을 해야 한다(ex. tensor board)
# fine tuning (kcbert, kobert 같은 모델들을 들고와서 갖고 있는 데이터에 알맞게 바꾼다)
# deep tech

# kobert, kcbert 돌아가는 내용 알아보기

# SGD, GD의 차이 (멘토님 면접질문)
# 쓰는 모델에 대한 개념 숙지 필요

# 한빛미디어


In [28]:
import rich  # 출력을 이쁘게 꾸며주는 라이브러리
from tqdm.auto import tqdm  # 진행바 라이브러리
from rich.table import Table

table = Table(title="Model Comparison Table")
table.add_column("Model Name", justify="left", style="green")
table.add_column("Accuracy", justify="right")

for model_name, model in tqdm(model_pipes, leave=False):
    acc = return_kfold_accuarcy(model)
    table.add_row(model_name, f"{acc:0.3f}")

rich.print(table)

  0%|          | 0/2 [00:00<?, ?it/s]

  Model Comparison Table  
┏━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Model Name  ┃ Accuracy ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ naive_bayes │    0.656 │
│ naive_bayes │    0.656 │
└─────────────┴──────────┘

In [37]:
test

,date,content
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...
...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...


In [34]:
test = train.drop(['label'], axis=1)
test

,date,content
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...
...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...


In [41]:
from pycaret.classification import *

ModuleNotFoundError: No module named 'pycaret'

In [ ]:
train_ohe = pd.get_dummies(train)
test_ohe = pd.get_dummies(test)

In [ ]:
X = train.drop(['label'], axis=1)
y = train['label']
X_test = test_ohe.copy()

In [ ]:
import optuna

#XGB 하이퍼 파라미터들 값 지정
def objectiveXGB(trial,X,y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

    #https://xgboost.readthedocs.io/en/latest/parameter.html 참고
    param = {
    'verbosity':1,
    'objective':'reg:squarederror', #'reg:linear' 회귀
    'max_depth':trial.suggest_int('max_depth',3,30),
    'learning_rate':trial.suggest_loguniform('learning_rate',1e-8,1e-2),
    'n_estimators':trial.suggest_int('n_estimators',100,3000),
    'subsample':trial.suggest_loguniform('subsample',0.7,1),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 300 ),
    'alpha': trial.suggest_loguniform( 'alpha', 1e-3, 10.0),
    'random_state': 42}

#학습모델 생성
    model=XGBRegressor(**param)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=25,verbose=False)

    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    return mae

In [ ]:
studyXGB=optuna.create_study(direction='minimize')

# n_trials 지정
studyXGB.optimize(lambda trial: objectiveXGB(trial, X, y), n_trials=100) 

print('study.best_params:', studyXGB.best_trial.value)
print('Number of finished trials:', len(studyXGB.trials))
print('Best trial:', studyXGB.best_trial.params)
print('study.best_params:', studyXGB.best_params)
#파라미터 중요도 시각화
optuna.visualization.plot_param_importances(studyXGB)

In [ ]:
train

In [ ]:
train['date'] = train['date'].astype('category')
train['content'] = train['content'].astype('category')

models = [
    ("naive_bayes", BernoulliNB()),
    ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
    ("rfc", RandomForestClassifier(random_state=42, n_jobs=-1)),
    ("SVC", SVC(random_state=42)),
    ("ada", AdaBoostClassifier(random_state=42)),
    ("lgbm", LGBMClassifier(random_state=42)),
    ("lgbm2", LGBMClassifier(n_estimators=80, random_state=42)),
    ("xgb", XGBClassifier(random_state=42)),
    ("knc1", KNeighborsClassifier()),
    ("knc2", KNeighborsClassifier(n_neighbors=4))
]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

table = Table(title="Model Comparison Table")
table.add_column("Model Name", justify="left", style="green")
table.add_column("Accuracy", justify="right")

for model_name, model in tqdm(model_pipes, leave=False):
    acc = return_kfold_accuarcy(model)
    table.add_row(model_name, f"{acc:0.3f}")

rich.print(table)

In [ ]:
from sklearn.ensemble import StackingClassifier

stack_models = [(name, get_pipe(model, name)) for name, model in models]

stacking = StackingClassifier(stack_models)
acc = return_kfold_accuarcy(stacking)
rich.print(acc)